In [6]:
import os
from mongo_connect import load_client

client = load_client()
CURR_DIR = os.getcwd()

db = client.get_database('Cluster0')
col = db.get_collection('games_rating')





Database(MongoClient(host=['ac-2whhhnb-shard-00-00.oiowsz6.mongodb.net:27017', 'ac-2whhhnb-shard-00-01.oiowsz6.mongodb.net:27017', 'ac-2whhhnb-shard-00-02.oiowsz6.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-4yq3is-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x0000022FB4EF0DD0>), 'Cluster0')


In [32]:
import datetime
import calendar

curr_month = datetime.datetime.now().month
curr_day = datetime.datetime.now().day

months = []
for i in range(7): 
    months.append(curr_month)
    if curr_month != 1:
        curr_month -= 1
    else:
        curr_month = 12

day_count = []
for i in months: 
    num_days = calendar.monthrange(datetime.datetime.now().year, i)[1]
    day_count.append(num_days)
print(months)
print(day_count)
print(curr_day)

last_year = months[0] < months[-1]

if last_year: 
    six_mounth_ago = datetime.datetime(datetime.datetime.now().year - 1, months[-1], datetime.datetime.now().day)
else:
    six_mounth_ago = datetime.datetime(datetime.datetime.now().year, months[-1], datetime.datetime.now().day)

# gerer le cas ou le jour est 31 ou 30 et n'existe pas dans le mois d'il ya 6mois

print(six_mounth_ago)

[3, 2, 1, 12, 11, 10, 9]
[31, 29, 31, 31, 30, 31, 30]
10
2023-09-10 00:00:00
